In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
temp_df=pd.read_csv('IMDB Dataset.csv')

In [3]:
df=temp_df.iloc[:10000]

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [6]:
df['sentiment'].value_counts()

positive    5028
negative    4972
Name: sentiment, dtype: int64

In [7]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [8]:
df.duplicated().sum()

17

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
#basic preprocessing
#remove tags
#remove stopwords
#Lowercase

In [11]:
import re
def remove_tags(raw_text):
    cleaned_text=re.sub(re.compile('<.*?>'),'',raw_text)
    return cleaned_text

In [12]:
df['review']=df['review'].apply(remove_tags)

In [13]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [14]:
df['review']=df['review'].apply(lambda x:x.lower())

In [15]:
from nltk.corpus import stopwords
sw_list=stopwords.words('english')
df['review']=df['review'].apply(lambda x:[item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [16]:
#creating the independent and the dependent feature
X=df.iloc[:,0:1]
y=df['sentiment']

In [17]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y=encoder.fit_transform(y)

In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
print(x_train.shape)
print(x_test.shape)

(7986, 1)
(1997, 1)


In [19]:
#Applying Bow
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [20]:
x_train_bow=cv.fit_transform(x_train['review']).toarray()
x_test_bow=cv.transform(x_test['review']).toarray()

In [21]:
x_train_bow.shape

(7986, 48382)

In [22]:
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()
gnb.fit(x_train_bow,y_train)

GaussianNB()

In [23]:
y_pred=gnb.predict(x_test_bow)
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.6339509263895844
[[716 265]
 [466 550]]
              precision    recall  f1-score   support

           0       0.61      0.73      0.66       981
           1       0.67      0.54      0.60      1016

    accuracy                           0.63      1997
   macro avg       0.64      0.64      0.63      1997
weighted avg       0.64      0.63      0.63      1997



In [24]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(x_train_bow,y_train)
y_pred=rf.predict(x_test_bow)
accuracy_score(y_test,y_pred)

0.8532799198798198

In [25]:
cv=CountVectorizer(max_features=3000)
x_train_bow=cv.fit_transform(x_train['review']).toarray()
x_test_bow=cv.transform(x_test['review']).toarray()
rf=RandomForestClassifier()
rf.fit(x_train_bow,y_train)
y_pred=rf.predict(x_test_bow)
accuracy_score(y_test,y_pred)

0.8487731597396094

In [26]:
cv=CountVectorizer(ngram_range=(1,2),max_features=5000)
x_train_bow=cv.fit_transform(x_train['review']).toarray()
x_test_bow=cv.transform(x_test['review']).toarray()
rf=RandomForestClassifier()
rf.fit(x_train_bow,y_train)
y_pred=rf.predict(x_test_bow)
accuracy_score(y_test,y_pred)

0.8522784176264396

### Using Tfidf

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()

In [30]:
x_train_idf=tfidf.fit_transform(x_train['review']).toarray()
x_test_idf=tfidf.transform(x_test['review']).toarray()
rf=RandomForestClassifier()
rf.fit(x_train_idf,y_train)
y_pred=rf.predict(x_test_idf)
accuracy_score(y_test,y_pred)

0.8597896845267902